In [42]:
import os, json, re, string, requests, pprint, nltk, operator, numpy as np, math, tqdm

In [45]:
def process_judgment(judgment):
    data_to_query = (",").join(judgment)
    r = requests.post(data=data_to_query.encode("utf-8"), url="http://localhost:9200")
    response_text = r.text
    splited_response = response_text.splitlines()
    splited_response = [" ".join(x.replace("\t", " ").replace("none", "")[1:].split(":")[:2][:1]).replace(" ", ":")
                        for x in splited_response if ":" in x]
    return splited_response

In [75]:
all_words = []

def read_file(file_path):
    with open (file_path) as file:
        judgment = []
        json_content = json.load(file)
        item_count = 0
        for item in tqdm.tqdm(json_content):
            item_count += 1
            text_content = re.sub("<.*?>", "", item["textContent"])
            text_content = text_content.replace('-\n', '')
            word_content = text_content.split()
            topicSpecificPunctuation = '„”–§…«»'
            translator = str.maketrans('', '', string.punctuation+topicSpecificPunctuation)

            for word in word_content:
                word = word.translate(translator).lower()
                if len(word)>0:
                    judgment.append(word)
            unigrams = process_judgment(judgment)
                
            with open(file_path + "_results.txt", 'w') as out_file:
                for unigram in unigrams:
                    out_file.write(unigram + "\n")
            

def read_all_judgments_from_2018():
    with open("../data_filtered/raport.txt", "w") as raport_file:
        for filename in tqdm.tqdm(os.listdir("../data_filtered/")):
            if not filename + "_results.txt" in os.listdir("../data_filtered/"):
                raport_file.write("Writing to file " + filename)
                raport_file.flush()
                read_file("../data_filtered/" + filename)
                raport_file.write("Writing to file " + filename + "FINISHED")
                raport_file.flush()
        
read_all_judgments_from_2018()

 10%|█         | 10/100 [03:41<33:13, 22.15s/it]


KeyboardInterrupt: 

In [ ]:
def read_list_from_file(file_path):
    with open(file_path) as file:
        content = file.read().split("\n")
        return content

In [ ]:
splited_response = processed_unigrams
splited_response = [item for sublist in splited_response for item in sublist]


In [ ]:
bigrams = nltk.bigrams(splited_response)
unigrams_frequency = nltk.FreqDist(splited_response)
bigrams_frequency = nltk.FreqDist(bigrams)

In [ ]:
sorted_unigrams = sorted(unigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_unigrams[:30])

In [ ]:
sorted_bigrams = sorted(bigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_bigrams[:30])

In [ ]:
TOTAL_COUNT = len(splited_response)

In [ ]:
def shannon_entrophy(word_occurences, total_words):
    sum = 0
    for x in np.nditer(word_occurences):
        if x!= 0:
            sum += (x/total_words) * math.log(x/total_words)
    return sum

def H(word_occurences):
    return shannon_entrophy(word_occurences, TOTAL_COUNT)

def calculate_contingency_table(bigram, bigram_count, total_words):
    first, second = bigram[0], bigram[1]
    first_occurence, second_occurence = unigrams_frequency[first], unigrams_frequency[second]
    '''
    |------  |---------| 
    | A,B    |B,notA   |
    |------  |---------|
    | A,notB |notA,notB|
    |------|---------|
    '''
    return np.array([
            [bigram_count, first_occurence-bigram_count],
            [second_occurence-bigram_count, total_words-first_occurence-second_occurence-bigram_count]])

def log_likeliheood_ratio(bigram_key):
    k = calculate_contingency_table(bigram_key, bigrams_frequency[bigram_key], TOTAL_COUNT)
    return 2 * np.sum(k) * (H(k) - H(k.sum(axis=0)) -H(k.sum(axis=1)))

In [ ]:
log_ratios = [(key, log_likeliheood_ratio(key), value) for key, value in bigrams_frequency.items()]

In [ ]:
sorted_by_log_ratios = sorted(log_ratios, key=operator.itemgetter(1), reverse=True)
sorted_by_log_ratios[:30]

In [ ]:
filtered_by_tags = [log_ratio for log_ratio in log_ratios 
                    if "subst" in log_ratio[0][0] and
                    ("subst" in log_ratio[0][1] or "adj" in log_ratio[0][1])]
filtered_by_tags_sorted = sorted(filtered_by_tags, key=operator.itemgetter(1), reverse=True)
filtered_by_tags_sorted[:30]